In [3]:
from data_loader import load_data

In [68]:
import numpy as np
import random

# Greedy Başlangıç Çözümü
def greedy_initial_solution(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs):
    solution = np.zeros(n_customers)  # Her müşteri için hangi depoya atandığını tutacak
    remaining_capacities = depot_capacities.copy()  # Depoların kalan kapasiteleri

    for customer in range(n_customers):
        min_cost = float('inf')
        best_depot = -1

        # En uygun depo seçimi (kapasiteyi de dikkate alarak)
        for depot in range(n_depots):
            if remaining_capacities[depot] >= customer_demands[customer]:  # Yeterli kapasite var mı?
                cost = costs[depot, customer]  # Maliyet
                if cost < min_cost:
                    min_cost = cost
                    best_depot = depot
        
        if best_depot != -1:
            solution[customer] = best_depot
            remaining_capacities[best_depot] -= customer_demands[customer]
    
    return solution

# PSO Algoritması
def pso_algorithm(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs, greedy_solution):
    # Başlangıç çözümü olarak greedy çözümünü kullanıyoruz
        # PSO Parametreleri
    w = 0.9  # Atalet katsayısı
    c1 = 1.5  # Bireysel öğrenme katsayısı
    c2 = 1.5  # Sosyal öğrenme katsayısı
    n_particles = 50  # Partikül sayısı
    n_iterations = 100  # Iterasyon sayısı
    particles = np.array([greedy_solution for _ in range(n_particles)])  # İlk parçacıklar
    velocities = np.zeros_like(particles)  # Başlangıç hızları (0)
    
    # En iyi kişisel çözüm ve en iyi global çözümü başlangıç olarak greedy çözümüne ayarlıyoruz
    personal_best = np.copy(particles)
    personal_best_costs = np.array([calculate_cost(solution, n_depots, depot_capacities, setup_costs, customer_demands, costs) for solution in particles])
    
    global_best = personal_best[np.argmin(personal_best_costs)]
    global_best_cost = np.min(personal_best_costs)

    for iteration in range(n_iterations):
        for i in range(n_particles):
            # Hız güncellemeleri
            velocities[i] = (w * velocities[i] +
                             c1 * random.random() * (personal_best[i] - particles[i]) +
                             c2 * random.random() * (global_best - particles[i]))
            
            # Yeni pozisyon güncellemeleri
            particles[i] = particles[i] + velocities[i]

            # Çözümü doğrulama ve iyileştirme (kapasite kısıtlaması)
            particles[i] = np.clip(particles[i], 0, n_depots - 1)  # Depo dışı bir değere gitmesini engelle

            # Maliyet hesaplama
            cost = calculate_cost(particles[i], n_depots, depot_capacities, setup_costs, customer_demands, costs)

            # Kişisel en iyi güncelleme
            if cost < personal_best_costs[i]:
                personal_best[i] = particles[i]
                personal_best_costs[i] = cost

        # Global en iyi güncelleme
        min_cost_idx = np.argmin(personal_best_costs)
        if personal_best_costs[min_cost_idx] < global_best_cost:
            global_best = personal_best[min_cost_idx]
            global_best_cost = personal_best_costs[min_cost_idx]
        
        # `w` parametresinin zamanla azalması
        w = 0.9 - (0.5 * iteration / n_iterations)  # Atalet katsayısının azalması

    return global_best, global_best_cost

# Maliyet Hesaplama Fonksiyonu
def calculate_cost(solution, n_depots, depot_capacities, setup_costs, customer_demands, costs):
    total_cost = 0
    depot_usage = np.zeros(n_depots)

    for customer in range(n_customers):
        depot = int(solution[customer])  # Hangi depoya atandığı
        depot_usage[depot] += customer_demands[customer]  # Depoya yapılan talepleri takip et

    # Depoların kurulumu ve müşteri maliyetleri
    for depot in range(n_depots):
        if depot_usage[depot] > 0:
            total_cost += setup_costs[depot]  # Depo kurulumu maliyeti
            for customer in range(n_customers):
                if int(solution[customer]) == depot:
                    total_cost += costs[depot, customer]  # Müşteri-depo maliyeti

    return total_cost




In [51]:
# Veri setinizi yükleyin
filename = 'Dataset/wl_25'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

# Depo ve müşteri sayısını tekrar belirleyelim
n_depots = len(depot_capacities)  # Depo sayısı
n_customers = len(customer_demands)  # Müşteri sayısı

# costs matrisini numpy dizisine çevir
costs = np.array(costs)  # Müşteri-depo maliyetleri

# costs matrisinin boyutlarını kontrol et
print(f"Mevcut costs boyutu: {costs.shape}")
print(f"Beklenen costs boyutu: ({n_depots}, {n_customers})")

# costs matrisinin boyutları uyumsuzsa, uygun şekilde düzeltelim
if costs.shape[0] != n_depots or costs.shape[1] != n_customers:
    print("costs matrisinin boyutları uyumsuz!")
    print(f"Mevcut costs boyutu: {costs.shape}")
    print(f"Beklenen costs boyutu: ({n_depots}, {n_customers})")

    # costs matrisinin boyutlarını transpoze edelim
    if costs.shape[0] == n_customers and costs.shape[1] == n_depots:
        costs = costs.T  # Eğer satırlar ve sütunlar yer değişmişse, transpoz alıyoruz
        print("costs matrisini transpoze ettik.")
    elif costs.shape[0] != n_depots or costs.shape[1] != n_customers:
        print("costs matrisinin boyutları hala uyumsuz!")
        costs = costs[:n_depots, :n_customers]
        print("costs matrisini uygun boyutlarda kestik.")
    
    print(f"Yeni costs boyutu: {costs.shape}")
else:
    print("costs matrisinin boyutları uyumlu.")

# Başlangıç çözümünü greedy ile oluşturuyoruz
greedy_solution = greedy_initial_solution(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs)

# PSO ile çözümü iyileştiriyoruz
best_solution, best_cost = pso_algorithm(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs, greedy_solution)

# Sonuçları yazdır
print("En iyi çözüm:", best_solution)
print("En düşük maliyet:", best_cost)


Mevcut costs boyutu: (50, 25)
Beklenen costs boyutu: (25, 50)
costs matrisinin boyutları uyumsuz!
Mevcut costs boyutu: (50, 25)
Beklenen costs boyutu: (25, 50)
costs matrisini transpoze ettik.
Yeni costs boyutu: (25, 50)
En iyi çözüm: [ 7. 11.  0. 24. 20.  0.  1.  2. 21.  7.  3.  4.  5.  0.  6.  7.  3.  8.
 20.  6. 20.  9. 10.  0. 11.  4. 12. 10. 13. 14. 15. 10. 15. 16. 11. 11.
 17. 18. 23. 24. 19. 20. 21.  6. 22. 23. 23.  6. 24. 11.]
En düşük maliyet: 832291.15


In [65]:
filename = 'Dataset/wl_50'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

n_depots = np.array(n_depots)  # Depo sayısı
n_customers = np.array(n_customers)  # Müşteri sayısı
depot_capacities = np.array(depot_capacities)  # Depo kapasiteleri
setup_costs = np.array(setup_costs)  # Depo kurulum maliyetleri
customer_demands = np.array(customer_demands)  # Müşteri talepleri
costs = np.array(costs)  # Müşteri-depo maliyetleri

# Başlangıç çözümünü greedy ile oluşturuyoruz
greedy_solution = greedy_initial_solution(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs)

# PSO ile çözümü iyileştiriyoruz
best_solution, best_cost = pso_algorithm(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs, greedy_solution)

# Sonuçları yazdır
print("En iyi çözüm:", best_solution)
print("En düşük maliyet:", best_cost)

En iyi çözüm: [ 9.  9.  9. 20.  4.  9.  9. 20.  8.  9. 20. 20. 20. 13.  9.  9. 20. 20.
 20.  9. 20.  9. 22.  9.  9.  9.  8. 27.  9.  9.  9.  9.  9.  4.  9.  9.
  9.  8.  8.  9.  4. 20.  8.  9.  8.  8.  8. 47.  8.  4.]
En düşük maliyet: 147928.0875


In [66]:
filename = 'Dataset/wl_200'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

n_depots = np.array(n_depots)  # Depo sayısı
n_customers = np.array(n_customers)  # Müşteri sayısı
depot_capacities = np.array(depot_capacities)  # Depo kapasiteleri
setup_costs = np.array(setup_costs)  # Depo kurulum maliyetleri
customer_demands = np.array(customer_demands)  # Müşteri talepleri
costs = np.array(costs)  # Müşteri-depo maliyetleri

# Başlangıç çözümünü greedy ile oluşturuyoruz
greedy_solution = greedy_initial_solution(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs)

# PSO ile çözümü iyileştiriyoruz
best_solution, best_cost = pso_algorithm(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs, greedy_solution)

# Sonuçları yazdır
print("En iyi çözüm:", best_solution)
print("En düşük maliyet:", best_cost)

En iyi çözüm: [153. 158. 131. 188.  81.   0. 108.  63. 108. 103. 117. 188.   7.  41.
  45. 158.  41.  22.  86.  38. 169. 131. 131. 134. 167.  75. 169. 188.
 131.  45. 117.   3.   3. 131. 167.  44. 149.  70.  22. 188.  38. 158.
  66. 149. 165. 188.  66. 153.  81.  71. 138.  13.  34.  63. 134.   1.
  34.  66.  41. 136.  63.   0.  88.  75.   0. 188. 117. 188.  81. 155.
  38.  22.  66.  63.  70. 136.   3.  81.  66. 103. 149.  45.   0. 188.
 131. 169. 165.   3. 188.  86.  81.  45.  28.   7. 143.   1. 158.  28.
 158. 143.  28.  13. 136. 108.  75.  28.  28. 117.  41.  86. 117. 155.
  70. 108. 149. 167.  38.  70.  86.  71. 153. 149.  66.  38.  71.   3.
 108.  41.  38. 136.  13. 117. 138.  44. 165. 149.   1. 158.  81.  13.
 165. 108. 169.  75.  13.  86. 158.  34.  75.  41.   7.   0.  28.  28.
 155. 158. 165.  44.  44.  28. 143. 167.  34. 117.  63.  66.  70.  22.
  86.  66.   3.  22. 167.  75.  45.  75. 143.  22.  71. 169. 143. 108.
 158. 143.   7. 188. 158.   1.  38.  45.  88. 165. 167.  13. 10

In [61]:
filename = 'Dataset/wl_300'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

n_depots = np.array(n_depots)  # Depo sayısı
n_customers = np.array(n_customers)  # Müşteri sayısı
depot_capacities = np.array(depot_capacities)  # Depo kapasiteleri
setup_costs = np.array(setup_costs)  # Depo kurulum maliyetleri
customer_demands = np.array(customer_demands)  # Müşteri talepleri
costs = np.array(costs)  # Müşteri-depo maliyetleri

# Başlangıç çözümünü greedy ile oluşturuyoruz
greedy_solution = greedy_initial_solution(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs)

# PSO ile çözümü iyileştiriyoruz
best_solution, best_cost = pso_algorithm(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs, greedy_solution)

# Sonuçları yazdır
print("En iyi çözüm:", best_solution)
print("En düşük maliyet:", best_cost)

En iyi çözüm: [258. 241. 156. 215.  98.  96. 205. 252. 247.  66.  30. 205.  89. 156.
 156. 217.  24.  67. 117. 163. 245.  96. 271. 117. 247.  70.  12. 166.
  72. 214. 258.  35. 134. 221.  67. 134.  88. 111. 237. 101. 214. 150.
 155. 252. 292.   7. 150. 252.  35. 267. 150. 245. 217. 292. 101. 267.
  10. 133. 292. 248.  88. 248.  72. 112.   7.  24. 288. 184.  29. 156.
 247. 166. 237. 217. 113. 275.  70.  48.  39. 155. 294.  12. 252.  48.
  98. 184. 171. 133. 247.   7.  72.   7.  66.  70. 101. 155.  88. 258.
  14. 271. 101. 134. 241.  24. 134. 274. 294.  39. 101. 248. 184.  28.
  14. 133. 150. 209. 184.  14.  72. 275.  88.  96. 150. 223. 133. 258.
  67. 166. 292.  89. 111. 166. 111. 111. 267.  72. 294. 117. 174.  24.
  70. 112. 134.  72. 205. 219.  14. 221. 294. 248. 163. 275. 133.  88.
 219.  10.  14. 166.  14.   7.  30. 221. 205. 174. 133.  14. 214. 133.
  98.  89. 111.  14. 219. 252. 252.  72. 274. 267. 150.  88. 136. 215.
 223. 219. 111.  48. 155.  10. 258. 184. 133. 267. 237.  14.  3

In [38]:
filename = 'Dataset/wl_500'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

n_depots = np.array(n_depots)  # Depo sayısı
n_customers = np.array(n_customers)  # Müşteri sayısı
depot_capacities = np.array(depot_capacities)  # Depo kapasiteleri
setup_costs = np.array(setup_costs)  # Depo kurulum maliyetleri
customer_demands = np.array(customer_demands)  # Müşteri talepleri
costs = np.array(costs)  # Müşteri-depo maliyetleri

# Başlangıç çözümünü greedy ile oluşturuyoruz
greedy_solution = greedy_initial_solution(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs)

# PSO ile çözümü iyileştiriyoruz
best_solution, best_cost = pso_algorithm(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs, greedy_solution)

# Sonuçları yazdır
print("En iyi çözüm:", best_solution)
print("En düşük maliyet:", best_cost)

En iyi çözüm: [454. 184. 440. 429. 297. 163. 163.  80. 338. 321. 427. 352. 138. 197.
  38.  58. 290. 296.  69. 305. 461. 140.  47. 425. 424.  18. 400. 345.
  22. 302. 476. 454. 148.  18. 349.  22. 374. 390. 338.  39. 352.  41.
 345. 300.  76.  70. 477.  47. 277. 429. 461.  47. 337.  58.  82.  69.
 373. 426.  41.  38. 300. 426.  70. 459. 297. 253. 166. 321. 424. 321.
  78. 249. 425.  69. 321. 200. 176. 138. 184.   1.  37. 352. 415. 305.
 138.  80.  36.  36. 445. 163.  58. 370. 190. 349.  80. 206. 444.  76.
  37. 305. 401. 338. 376. 269. 373.  82. 179. 163. 297. 374. 302.  39.
 401. 190.  20. 249. 479.  18.  81. 179. 148. 305. 376. 400.  76. 305.
 202.   1. 376. 337. 401.  47. 362. 381. 363. 261. 337. 290. 154. 169.
 440.  36. 169. 300. 200. 363. 400.  41. 176. 381.  22.  63. 373. 486.
 486. 381. 206. 163. 305. 454. 400. 370. 362.  41. 190.  69.  69. 370.
  70. 166. 305. 477. 202. 321. 138. 479. 321. 415. 349. 300. 297. 459.
 269.  36.  18. 296. 296.  76.   1.  41.  82. 444. 479. 390. 29